In [1]:
from model import ML_Agent
from pso import PSO
from ga import GA
from bees import BEES
import pandas as pd
import numpy as np
from scipy import stats
from utility import split_data # (dataframe, label, do_shuffle=True, test_size=0.1):

In [2]:
df = pd.read_csv('thyroid.csv')
# thyroid.csv
df = df.sample(frac=1)
df.head(5)

,CLASS,T3RESIN,THYROXIN,THYRONINE,THYROID,TSH_VALUE
34,1,117,12.2,1.9,1.2,3.9
150,2,139,16.4,3.8,1.1,-0.2
193,3,120,1.9,0.7,18.5,24.0
43,1,121,13.5,1.5,1.6,0.5
71,1,120,12.4,2.4,0.8,1.9


In [3]:
train, validation, test = split_data(df, 'CLASS')
print(train[:5], train.shape)
print(validation[:5], validation.shape)
print(test[:5], test.shape)

   CLASS  T3RESIN  THYROXIN  THYRONINE  THYROID  TSH_VALUE
0      1      110      10.4        1.6      1.6        2.7
1      1      101       6.7        1.3      1.0        5.7
2      1      101       7.1        2.2      0.8        2.2
3      1      116      10.1        2.2      1.6        0.8
4      1      100       9.5        2.5      1.3       -0.2 (172, 6)
   CLASS  T3RESIN  THYROXIN  THYRONINE  THYROID  TSH_VALUE
0      1      112      10.6        1.6      0.9       -0.1
1      1      129      11.9        2.7      1.2        3.5
2      1      102       9.5        1.4      1.1        1.6
3      1      108       7.1        1.3      1.6        2.2
4      1      118      12.2        1.5      1.0        2.3 (21, 6)
   CLASS  T3RESIN  THYROXIN  THYRONINE  THYROID  TSH_VALUE
0      1      113       9.0        2.0      1.8        1.6
1      1      111       8.5        1.6      1.1        3.9
2      1      109       5.3        1.6      1.4        1.5
3      1      105       7.0        1.5 

In [4]:
def preprocessing(dataframe):
    scaled_dataframe = pd.DataFrame()
    for col in dataframe.columns:
        data_col = dataframe[col]
        scaled_dataframe[col] = stats.zscore(data_col)
    return scaled_dataframe

def one_hot_encoding(Target, n_classes):
    encoded_array = list()
    unique_label = np.unique(Target)
    rules = dict()
    for idx, label in enumerate(unique_label, 0):
        rules[label] = idx
    for target in Target:
        arr = np.zeros(n_classes)
        arr[rules[target]] = 1
        encoded_array.append(arr)
    return pd.DataFrame(np.array(encoded_array))

In [5]:
x_train, y_train = train.drop('CLASS', axis=1), train['CLASS']
x_val, y_val = validation.drop('CLASS', axis=1), validation['CLASS']
x_test, y_test = test.drop('CLASS', axis=1), test['CLASS']

In [6]:
n_class = len(np.unique(y_train))
x_train, y_train = preprocessing(x_train), one_hot_encoding(y_train, n_class)
x_val, y_val = preprocessing(x_val), one_hot_encoding(y_val, n_class)
x_test, y_test = preprocessing(x_test), one_hot_encoding(y_test, n_class)

In [7]:
bees = BEES(dict(name='pso_agent1', level='weak ai', m=100, weight_constraint=[-1.,1.], \
               c1=3, c2=0.5, c3=3., max_c1=3., min_c1=0.5, max_c2=3., min_c2=0.5, max_c3=3., min_c3=0.5, \
               velocity_constraint=[-0.05, 0.05], w=0.05, w_decay=0.99, goal=0.05, \
              initializer='uniform'))
ga = GA(dict(name='ga_agent2', level='weak weak ai', m=30, x_rate=[0.1, 0.95], m_rate=[0.05, 0.2], weight_constraint=[-1.,1.], \
            radioactive_rating=[0.1,1.0], grow_factor=[0.1, 1.0]))
model = ML_Agent(n_feature=5, target_size=3)

building object according to  {'name': 'pso_agent1', 'level': 'weak ai', 'm': 100, 'weight_constraint': [-1.0, 1.0], 'c1': 3, 'c2': 0.5, 'c3': 3.0, 'max_c1': 3.0, 'min_c1': 0.5, 'max_c2': 3.0, 'min_c2': 0.5, 'max_c3': 3.0, 'min_c3': 0.5, 'velocity_constraint': [-0.05, 0.05], 'w': 0.05, 'w_decay': 0.99, 'goal': 0.05, 'initializer': 'uniform'}


In [8]:
# add_layer(self, units, activation='relu', regularizer='l2', use_bias=True):
model.add_layer(32, 'relu', False)
model.add_layer(8, 'relu', False)
model.add_layer(3, 'sigmoid', True)

In [9]:
model.compile_configuration(bees, loss='categorical_crossentropy', \
                            monitor=['accuracy', 'precision', 'recall', 'essential_metrics'], regularizer='l2') # 

In [ ]:
# x, y, max_iter=100, goal=0.1, batch_size=5, validation=False, x_val=[], y_val=[], validation_batch_size=5
minimum_loss = model.fit(x_train, y_train, max_iter=100, goal=0.001, batch_size=10, validation=True, x_val=x_val, y_val=y_val, validation_batch_size=5)
print('minimum loss:', minimum_loss)

Iteration: 0
update global [3.3875248]
update global [3.3863034]
update global [3.3854096]
update global [3.3849573]
update global [3.3820353]
Update Pool
Update Pool
update local, [3.1962464]
update global [3.1962464]
update local, [3.2988577]
update local, [3.3037996]
update local, [3.2979274]
update local, [3.310649]
update local, [3.3095858]
update local, [3.3014987]
update local, [3.3020551]
update local, [3.3040438]
update local, [3.3008306]
update local, [3.3006768]
update local, [3.3025293]
update local, [3.2979102]
update local, [3.1941724]
update global [3.1941724]
update local, [3.2951107]
update local, [3.3006115]
update local, [3.198133]
update local, [3.2047043]
update local, [3.299852]
update local, [3.3003783]
update local, [3.2964342]
update local, [3.3065145]
update local, [3.2079468]
update local, [3.2971587]
update local, [3.302912]
update local, [3.2957664]
update local, [3.3063862]
update local, [3.3038247]
update local, [3.3036654]
update local, [3.3045242]
updat

update local, [2.6100645]
update local, [2.5497031]
update local, [2.481405]
update global [2.481405]
update local, [2.5019193]
update local, [2.500256]
update local, [2.5762284]
update local, [2.511302]
update local, [2.5438635]
update local, [2.4960988]
update local, [2.471557]
update global [2.471557]
update local, [2.4696593]
update global [2.4696593]
update local, [2.4920986]
update local, [2.52767]
update local, [2.4471147]
update global [2.4471147]
update local, [2.8953128]
update local, [2.8593712]
update local, [2.4282362]
update global [2.4282362]
update local, [2.5220735]
update local, [2.5568333]
update local, [2.4878979]
update local, [2.466273]
update local, [2.584081]
update local, [2.5233698]
update local, [2.4904654]
update local, [2.5034482]
update local, [2.502524]
update local, [2.375481]
update global [2.375481]
update local, [2.5807872]
update local, [2.452498]
update local, [2.4917598]
update local, [2.4975507]
update local, [2.497269]
update local, [2.5741751]
u

Update Pool
update local, [1.1239775]
update global [1.1239775]
update local, [1.0075207]
update global [1.0075207]
update local, [1.0419041]
update local, [1.6371199]
update local, [1.7676319]
update local, [1.6269028]
update local, [1.8000023]
update local, [1.9217454]
update local, [2.011144]
update local, [1.7554278]
update local, [1.1530735]
update local, [1.4471754]
update local, [0.91547155]
update global [0.91547155]
update local, [1.8489038]
update local, [1.6257619]
update local, [1.8910568]
update local, [1.9091313]
update local, [1.7678809]
update local, [2.083827]
update local, [1.2494316]
update local, [1.6420757]
update local, [2.021574]
update local, [1.147081]
update local, [1.9120435]
update local, [0.9960169]
update local, [1.0123944]
update local, [1.045356]
update local, [1.7629749]
update local, [1.9854076]
update local, [1.9387555]
update local, [1.851118]
update local, [1.8352972]
update local, [1.0221773]
update local, [1.7421397]
update local, [1.0343344]
upda

update local, [1.2078041]
update local, [1.2517929]
Update Pool
Update Pool
update local, [0.88503796]
update global [0.88503796]
Update Pool
update local, [1.3082922]
update local, [1.3012316]
update local, [1.4984989]
update local, [1.3056126]
Update Pool
Update Pool
update local, [1.0465043]
update local, [1.2028058]
update local, [1.2611387]
update local, [1.2377098]
update local, [1.1123451]
update local, [1.0115849]
update local, [1.194423]
update local, [1.1330355]
update local, [1.1502377]
update local, [1.0688347]
update local, [1.3975137]
update local, [1.0687389]
update local, [1.2236882]
update local, [1.1833885]
update local, [1.3887585]
Update Pool
Update Pool
Update Pool
no improvement from base [0.94850814]

Itr: 1
C1: 0.464524765729585, C2:0.07095046854082998, C3:0.464524765729585
best:[0.88503796], overal:22.07536865234375
validation:[0.83629835]
Iteration: 2
update local, [0.59097743]
update global [0.59097743]
update local, [0.3391725]
update global [0.3391725]
upda

no improvement from base [0.09704053]

Itr: 3
C1: 0.4601889338731444, C2:0.07962213225371119, C3:0.4601889338731444
best:[0.0283131], overal:0.4432052230834961
validation:[0.09704053]
Iteration: 4
Update Pool
Update Pool
Update Pool
Update Pool
Update Pool
Update Pool
Update Pool
Update Pool
Update Pool
Update Pool
Update Pool
Update Pool
Update Pool
Update Pool
Update Pool
Update Pool
Update Pool
Update Pool
no improvement from base [0.09704053]

Itr: 4
C1: 0.45799457994579945, C2:0.08401084010840108, C3:0.45799457994579945
best:[0.0283131], overal:1.4384123229980468
validation:[0.02693704]
Iteration: 5
Update Pool
Update Pool
Update Pool
Update Pool
Update Pool
Update Pool
Update Pool
Update Pool
Update Pool
Update Pool
update local, [0.5609709]
update local, [0.5443081]
update local, [0.7675778]
update local, [0.52433753]
update local, [0.56103003]
update local, [0.621735]
Update Pool
Update Pool
Update Pool
Update Pool
Update Pool
Update Pool
Update Pool
Update Pool
no improvement 

In [ ]:
loss, board = model._evaluate(x_test, y_test, len(y_test))
print('loss', loss)
print(board)
y_pred = model._predict(x_test)
y_test = np.array(y_test)
loss = model.__loss__(y_pred, y_test)
print(loss)

In [ ]:
from metrics import MSE
mse_loss = MSE(y_pred, y_test)
print(mse_loss)

In [ ]:
for name, met in board.items():
    if type(met) == dict:
        print(name)
        for key, value in met.items():
            print(key, value)
    else:
        print(name, met)

In [ ]:
i = 0
for pred, true in zip(y_pred, y_test):
    print(pred, 'VS', true)
    i += 1

In [ ]:
for idx, particle in enumerate(bees.swams):
    print(idx, particle.fitness)

In [ ]:
wt = model.flat_weight
for w in wt:
    print(w)

In [ ]:
layer_weight = model.weight_matrix
layer_bias = model.bias_matrix
for weights in layer_weight:
    print(weights)
print(layer_bias)

In [ ]:
print(wt.shape)